In [1]:
import os
import sys
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
import tomopy

path = "/Users/hojunekim/Desktop/data_preprocessing/"
os.chdir(path)
!pwd

/Users/hojunekim/Desktop/data_preprocessing


In [6]:
def convert_nib_to_npy(nib_file_path, npy_file_path):
    # Load .nib file
    img = nib.load(nib_file_path)
    
    # Extract data from the loaded image
    data = img.get_fdata()
    
    # Change (512,512,70) format to (70,512,512) format
    data = data.transpose((2, 0, 1))
    
    # Save data as .npy file
    np.save(npy_file_path, data)
    
    print(f"Successfully converted {nib_file_path} to {npy_file_path}.")

def plot_input_images(npy_file_path):
    data = np.load(npy_file_path)
    num_images = data.shape[0]
    num_rows = num_images // 5 if num_images % 5 == 0 else (num_images // 5) + 1
    fig, axes = plt.subplots(num_rows, 5, figsize=(10, 2 * num_rows))

    for i, ax in enumerate(axes.flat):
        if i >= num_images:
            break

        slice_img = data[i, :, :]
        ax.imshow(slice_img, cmap='gray')
        ax.set_title(f"Slice: {i}")
        ax.axis('off')

    plt.tight_layout()

    # Save the plot as PNG file
    file_name = os.path.splitext(os.path.basename(npy_file_path))[0]
    plt.savefig(f"figures/plot of {file_name}.png")
    plt.show()

def create_sinogram(img_stack, theta, pad=True):
    proj = tomopy.project(img_stack, theta, center=None, emission=True, pad=pad, sinogram_order=False)
    proj = proj.transpose((0, 1, 2))  # Adjust the transpose order
    return proj

def plot_sinogram_images(sinogram, num_images_to_plot=5):
    interval = sinogram.shape[0] // num_images_to_plot

    if num_images_to_plot%3 == 0:
        rows = (num_images_to_plot // 3)
    else:
        rows = (num_images_to_plot // 3) + 1
    
    fig, axes = plt.subplots(rows, 3, figsize=(12, 6 * rows))

    for i, ax in enumerate(axes.flat):
        index = i * interval
        if index >= sinogram.shape[0]:
            break

        image = sinogram[index, :, :]
        ax.imshow(image, cmap='gray')
        ax.set_title(f"Index: {index}")
        ax.axis('off')

    plt.tight_layout()
    plt.show()

In [30]:
# Convert .nib file to .npy file
nib_file_path = 'img1.nii'
file_name = os.path.splitext(os.path.basename(nib_file_path))[0]
npy_file_path = 'f{file_name}.npy'
convert_nib_to_npy(nib_file_path, npy_file_path)

# Plot input images
plot_input_images(npy_file_path)

# Create sinogram
theta = np.arange(0, 180, 1)
slices = np.load('image_slices.npy')
print("input shape:", slices.shape)  # Input shape: (512, 512, 70)
sino = create_sinogram(slices, theta, pad=False)
print("output shape:", sino.shape)  # Output shape: (180, 70, 512)

# Save sinogram as .npy file
sinogram_file_path = 'f{file_name}_sinogram.npy'
np.save(sinogram_file_path, sino)

# Plot sinogram images
plot_sinogram_images(sino,18)

Successfully converted img1.nii to image_slices.npy.


In [ ]:
# Input : nib_file_path = data/CT_1.nii, theta, pad=True, only_sinogram=False
# If only_sinogram=True, then it will only create sinogram and save it as .npy file
# If only_sinogram=False, then it will save plot and .npy file for both sinogram and its original image

def create_sinogram(nib_file_path, theta, only_sinogram=True, pad=True):
    if only_sinogram:
        img = nib.load(nib_file_path)
        data = img.get_fdata()
        data = data.transpose((2, 0, 1))
        proj = tomopy.project(data, theta, center=None, emission=True, pad=pad, sinogram_order=False)
        sino = proj.transpose((0, 1, 2))  # Adjust the transpose order
        print("output shape:", sino.shape)  # Output shape: (180, 70, 512)
        sinogram_file_path = f"data/{file_name}_sinogram.npy"
        np.save(sinogram_file_path, sino)
    else:
        # Load .nib file and save it as .npy file
        img = nib.load(nib_file_path)
        file_name = os.path.splitext(os.path.basename(nib_file_path))[0]
        data = img.get_fdata()
        data = data.transpose((2, 0, 1))
        npy_file_path = f"data/{file_name}.npy"
        np.save(npy_file_path, data)
        print(f"Successfully converted {nib_file_path} to {npy_file_path}.")
        

        # Plot input imgaes and save plot as PNG file
        num_images = data.shape[0]
        num_rows = num_images // 5 if num_images % 5 == 0 else (num_images // 5) + 1
        fig, axes = plt.subplots(num_rows, 5, figsize=(10, 2 * num_rows))
        for i, ax in enumerate(axes.flat):
            if i >= num_images:
                break
            slice_img = data[i, :, :]
            ax.imshow(slice_img, cmap='gray')
            ax.set_title(f"Slice: {i}")
            ax.axis('off')
        plt.tight_layout()
        file_name = os.path.splitext(os.path.basename(npy_file_path))[0]
        plt.savefig(f"figures/plot of {file_name}.png")
        plt.show()


        # Create sinogram and save it as .npy file
        print("input shape:", data.shape)  # Input shape: (512, 512, 70)
        proj = tomopy.project(data, theta, center=None, emission=True, pad=pad, sinogram_order=False)
        sino = proj.transpose((0, 1, 2))  # Adjust the transpose order
        print("output shape:", sino.shape)  # Output shape: (180, 70, 512)
        sinogram_file_path = f"data/{file_name}_sinogram.npy"
        np.save(sinogram_file_path, sino)
        

        # Plot sinogram images and save the plot as PNG file
        num_images_to_plot = 18
        interval = sino.shape[0] // num_images_to_plot

        if num_images_to_plot%3 == 0:
            rows = (num_images_to_plot // 3)
        else:
            rows = (num_images_to_plot // 3) + 1
        
        fig, axes = plt.subplots(rows, 3, figsize=(12, 6 * rows))

        for i, ax in enumerate(axes.flat):
            index = i * interval
            if index >= sino.shape[0]:
                break

            image = sino[index, :, :]
            ax.imshow(image, cmap='gray')
            ax.set_title(f"Index: {index}")
            ax.axis('off')

        plt.tight_layout()
        plt.savefig(f"figures/sinogram of {file_name}.png")
        plt.show()

if __name__ == "__main__":
    # Extract command-line arguments
    if len(sys.argv) < 4:
        print("Usage: python sino.py <nib_file_path> <only_sinogram> <pad>")
        sys.exit(1)

    # Assign command-line arguments to variables
    nib_file_path = sys.argv[1]
    only_sinogram = bool(sys.argv[2])
    pad = bool(sys.argv[3])

    # Additional code remains the same
    theta = np.arange(0, 180, 1)
    create_sinogram(nib_file_path, theta, only_sinogram, pad)